
## 🧠 Using Neurosurfer in Jupyter — A Quick Start Guide

Welcome to **Neurosurfer**, a unified framework for building **intelligent, tool-augmented LLM agents** — locally or in the cloud.  
This notebook walks you through practical examples: loading models, wiring tools, building retrieval (RAG) pipelines, and streaming reasoning in real time — all using the same simple Python API.

Think of Neurosurfer as a **developer-friendly bridge** between:
- 🧩 **OpenAI-style APIs** and **local Transformers**
- 🔧 **Custom tools** and **autonomous agents**
- 📚 **Retrieval-augmented generation (RAG)** and **multi-model orchestration**

> **Requirements:**  
> Python 3.10+, `pip`, and optionally a GPU (for local Transformer backends).  
> Internet access is only needed when using hosted APIs like OpenAI.

---

### ⚙️ Installation

You can install Neurosurfer from PyPI — either a minimal setup for lightweight use, or a full install for local model support.

```bash
# Core install: FastAPI backend, agents, tools, and RAG
pip install neurosurfer

# For local LLMs (Transformers, Unsloth, vLLM, etc.)
pip install "neurosurfer[torch]"
```

Or clone and install from source if you want to explore or contribute:

```bash
git clone https://github.com/NaumanHSA/neurosurfer.git
cd neurosurfer
pip install -e .
```

Once installed, you can seamlessly switch between OpenAI models and local LLMs — no code changes required.



### 🔄 Enable Auto-Reload and Set Up Project Path

This setup cell ensures your notebook always runs with the **latest version** of your local Neurosurfer code — ideal when you’re actively editing the library.

- `%load_ext autoreload` and `%autoreload 2` automatically reload any modified modules before executing a new cell, so you don’t have to restart the kernel after every code change.
- The `sys.path.append(root_dir)` line adds the **project root directory** to Python’s import path, allowing you to import `neurosurfer` modules directly even when running this notebook from a subfolder (like `tutorials/`).

This is a standard pattern for working on notebooks inside a development repo while keeping imports clean and relative to the package root.


In [3]:
%load_ext autoreload
%autoreload 2

import sys, os

# Project Folder Structure
# tutorials/
# ├── basic_usage.ipynb/
# neurosurfer/
# ├── agents/
# │   ├── ...
# ├── tools/
# │   ├── ...
# └── ...

# Go up one directory from `tutorials/` to project root
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_dir)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 💡 Loading a Local LLM with Neurosurfer

**Neurosurfer** unifies **OpenAI-style** and **local transformer models** behind the same, simple API — whether you're running a large language model on your own GPU or calling a cloud-hosted one.

This cell shows how to initialize a **local transformer model** using the `TransformersModel` interface — the same API used by OpenAI models.  

Here we load **Qwen3-8B (4-bit quantized)** through the Unsloth backend. Neurosurfer automatically handles model loading, device selection (GPU/CPU), and unified prompting logic.  

- `model_name` points to your local model folder.  
- `max_seq_length` defines the maximum context window (useful for long documents).  
- `load_in_4bit=True` enables **quantized inference**, reducing GPU memory usage.  
- `enable_thinking` toggles reasoning visualization in certain modes.  
- `stop_words` defines when generation should halt.  

You can replace this local model with an **OpenAI model** by uncommenting the lower section — the rest of your prompt code stays identical, thanks to Neurosurfer’s **unified `.ask()` API**.



In [4]:
import logging
from neurosurfer.models.chat_models.transformers import TransformersModel
from neurosurfer import config 
import torch
torch.cuda.empty_cache()


MODEL_PARAMS = dict({
    "model_name": "/home/nomi/workspace/Model_Weights/Qwen3-8B-unsloth-bnb-4bit",
    "max_seq_length": 16_000,
    "load_in_4bit": True,
    "enable_thinking": False,  # main_gpu interpretation
    "verbose": False
})

LOGGER = logging.getLogger()
LLM = TransformersModel(
    **MODEL_PARAMS,
    stop_words=["Observation:"],
    logger = logging.getLogger(),
)

# Optionally, we can use OpenAI models
# from neurosurfer.models.chat_models.openai import OpenAIModel

# # Use an env var like OPENAI_API_KEY or configure your backend in the model init.
# llm = OpenAIModel(model_name="gpt-4o-mini")

# resp = llm.ask(
#     system_prompt="You are concise.",
#     user_prompt="In one sentence, what is Neurosurfer?",
#     max_new_tokens=128,
#     temperature=0.2,
#     stream=False,
# )
# print(resp.choices[0].message.content)


╔══════════════════════════════════════════════════════════════════╗
║                                                                  ║
║ ▓▓▓▓▓   ▓▓▓▓                                  ▓▓▓                ║
║  ▓▓ ▓▓   ▓▓  ▓▓▓▓ ▓  ▓ ▓ ▓ ▓▓▓▓ ▓▓▓ ▓  ▓ ▓ ▓  ▓   ▓▓▓▓ ▓ ▓       ║
║  ▓▓  ▓▓  ▓▓  ▓▁▁▓ ▓  ▓ ▓▓▏ ▓  ▓ ▓▁  ▓  ▓ ▓▓▏ ▓▓▓  ▓▁▁▓ ▓▓        ║
║  ▓▓   ▓▓ ▓▓  ▓    ▓  ▓ ▓   ▓  ▓   ▓ ▓  ▓ ▓    ▓   ▓    ▓         ║
║ ▓▓▓▓   ▓▓▓▓▓ ▓▓▓▓ ▓▓▓▓ ▓   ▓▓▓▓ ▓▓▓ ▓▓▓▓ ▓    ▓   ▓▓▓▓ ▓         ║
║ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ║
║ Orchestrate Agents - RAG - SQL Tools - Multi-LLM - FastAPI Ready ║
║ Faster builds, clearer flows, production-first                   ║
║                                                                  ║
║ Version: 0.1.3 | Python: 3.11.13                                 ║
║ OS: Linux 6.14.0-33-generic (x86_64)                             ║
║ Torch: 2.7.1+cu126   CUDA: yes (12.6)                            ║
║ MPS: no (built: False)          

/home/nomi/anaconda3/envs/LLMs/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO     | 2025-11-06 11:33:23 | config.py:<module> | PyTorch version 2.7.1+cu126 available.


/home/nomi/anaconda3/envs/LLMs/lib/python3.11/importlib/__init__.py:126: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  return _bootstrap._gcd_import(name[level:], package, level)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 11-06 11:33:24 [__init__.py:241] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO     | 2025-11-06 11:33:38 | transformers.py:init_model | Initializing Transformers model.
WARNING  | 2025-11-06 11:33:38 | transformers.py:init_model | Model is already quantized. Ignoring load_in_4bit=True.
INFO     | 2025-11-06 11:33:38 | modeling.py:get_balanced_memory | We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.78s/it]

INFO     | 2025-11-06 11:33:44 | transformers.py:init_model | Transformers model initialized successfully.


### 🗣️ Streaming Responses in Real Time

Here we send a simple prompt to the model — with a **system role** (“You are a joker”) and a **user message** (“Tell me a light-hearted joke.”).  

Setting `stream=True` enables **token-by-token streaming**, letting you display output as it’s being generated — just like ChatGPT’s live typing effect.  

- `LLM.ask(...)` uses the same API whether your backend is local (`TransformersModel`) or cloud (`OpenAIModel`).  
- `system_prompt` sets the model’s behavior/persona.  
- `user_prompt` is the actual query.  
- The `for chunk in stream_response:` loop iterates over streamed tokens, printing them as they arrive.  

This setup is ideal for interactive demos or notebooks where you want to **see the model “think” in real time**.


In [5]:
from IPython.display import display, Markdown, clear_output

system_prompt = "You are a joker."
user_prompt = """Tell me a light-hearted joke."""

stream_response = LLM.ask(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    stream=True
)

response = ""
md_display = display(Markdown(""), display_id=True)
for chunk in stream_response:
    chunk = chunk.choices[0].delta.content or ""
    response += chunk
    md_display.update(Markdown(response))

Why don't skeletons fight each other?  
Because they don't have the *guts*! 😄

## 🧩 Building a RAG Knowledge Base from the Codebase

This cell demonstrates how to **ingest your project repository** into a **Retrieval-Augmented Generation (RAG)** pipeline using Neurosurfer’s modular components.  

We’re turning the entire `neurosurfer` source directory into a searchable knowledge base that future agents can reference when reasoning or generating code.

Here’s what each part does:

- [**`SentenceTransformerEmbedder`**](https://naumanhsa.github.io/neurosurfer/api-reference/models/embedders/sentence-transformer/) — converts text chunks into dense vector embeddings using a lightweight model (`intfloat/e5-small-v2`).  
- [**`ChromaVectorStore`**](https://naumanhsa.github.io/neurosurfer/api-reference/vectorstores/chroma/) — stores those embeddings locally for fast semantic search.  
- [**`RAGIngestor`**](https://naumanhsa.github.io/neurosurfer/api-reference/rag/ingestor/) — orchestrates reading, chunking, embedding, and storing documents in one unified pipeline.  
- [**`Chunker`**](https://naumanhsa.github.io/neurosurfer/api-reference/rag/chunker/) — splits large files into manageable semantic segments.  
- [**`FileReader`**](https://naumanhsa.github.io/neurosurfer/api-reference/rag/filereader/) — automatically detects and reads supported file types (Markdown, Python, text, etc.).  

Finally:
```python
ing.add_directory(root_dir)
print(ing.build())
```

scans the neurosurfer/ directory, processes all files, and builds the vector index.
You can later query this index through your agents or retrieval tools — enabling the model to “understand” its own codebase.

In [6]:
# scripts/index_repo_for_rag.py
from pathlib import Path
from neurosurfer.rag.ingestor import RAGIngestor
from neurosurfer.rag.chunker import Chunker
from neurosurfer.rag.filereader import FileReader
from neurosurfer.vectorstores.chroma import ChromaVectorStore
from neurosurfer.models.embedders.sentence_transformer import SentenceTransformerEmbedder

# Initialize embedder 
embedder = SentenceTransformerEmbedder("intfloat/e5-small-v2")

# Initialize vector store using Chroma
vs = ChromaVectorStore(collection_name="neurosurf-repo")

# Initialize RAGIngestor which ingests the repo and stores it in the vector store 
ing = RAGIngestor(
    embedder=embedder,
    vector_store=vs, 
    chunker=Chunker(), 
    file_reader=FileReader(),
    default_metadata={"collection": "neurosurf"}
)

# Add directory to the ingestor. This will ingest all files in the directory and store it in the vector store 
root_dir = Path(os.getcwd()).parent.joinpath("neurosurfer")
ing.add_directory(root_dir)  # the repo root
ing.build()


INFO     | 2025-11-06 11:47:15 | SentenceTransformer.py:__init__ | Use pytorch device_name: cuda:0
INFO     | 2025-11-06 11:47:15 | sentence_transformer.py:__init__ | SentenceTransformer embedding model initialized.
INFO     | 2025-11-06 11:47:15 | posthog.py:__init__ | Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
[Init] ChromaVectorStore initialized with collection: neurosurf-repo


Batches: 100%|██████████| 1/1 [00:00<00:00, 12.32it/s]


{'status': 'ok',
 'sources': 99,
 'chunks': 608,
 'unique_chunks': 608,
 'added': 608,
 'finished_at': 1762415237.9988394}

### 🔍 Creating a RAG Agent for Context-Aware Answers

Now that the repository has been indexed, we can build a **RAGAgent** — a retrieval-augmented agent that can search, read, and reason over your stored knowledge base.  

Here’s how it fits together:

- [**`ChromaVectorStore`](https://naumanhsa.github.io/neurosurfer/api-reference/vectorstores/chroma/) reconnects to the same database we created earlier (`neurosurf-repo`), giving the agent instant access to embedded documents.  
- [**`SentenceTransformerEmbedder`](https://naumanhsa.github.io/neurosurfer/api-reference/models/embedders/sentence-transformer/) ensures new queries are embedded in the same vector space.  
- [**`LLM`](https://naumanhsa.github.io/neurosurfer/api-reference/models/chat-models/base-model/) handles the reasoning and response generation.  
- [**`RAGAgentConfig(top_k=6)`](https://naumanhsa.github.io/neurosurfer/api-reference/agents/rag-agent/) tells the agent to retrieve the top 6 most relevant chunks per query.  

This setup allows your model to **ground its answers** in your project’s actual code and documentation — effectively letting it *“read before it replies.”*  
Switching to an OpenAI backend (via `OpenAIModel`) is as simple as uncommenting that line — Neurosurfer keeps the agent logic unified.


In [7]:
# scripts/rag_helper.py
from neurosurfer.agents.rag import RAGAgent, RAGAgentConfig
from neurosurfer.vectorstores.chroma import ChromaVectorStore
from neurosurfer.models.embedders.sentence_transformer import SentenceTransformerEmbedder
from neurosurfer.models.chat_models.openai import OpenAIModel

vs = ChromaVectorStore(collection_name="neurosurf-repo")
# embedder = SentenceTransformerEmbedder("intfloat/e5-small-v2")
# llm = OpenAIModel(model_name="gpt-4o-mini")
rag_agent = RAGAgent(
    llm=LLM, 
    vectorstore=vs, 
    embedder=embedder, 
    config=RAGAgentConfig(top_k=6)
)

[Init] ChromaVectorStore initialized with collection: neurosurf-repo


### 🧠 Querying Your Codebase with the RAG Agent

With the **RAGAgent** ready, we can now ask natural-language questions about the code — and the agent will ground its responses in the ingested repository.  

There are two key modes you can use:
- **`retrieve()`** (commented out) — fetches the most relevant text chunks from the vector store without generating an answer. Useful for inspection, debugging or using the retrieved chunks as context for another LLM.  
- **`run()`** — performs the full RAG cycle: retrieves context → augments the prompt → streams an LLM-generated answer.  

In this example, we ask:
```python
"Explain Chunker Configurations in table form."
```

The model retrieves relevant code and docstrings from the repo, then explains them clearly — in this case as a formatted Markdown table.

The display block (`md_display.update`) allows the answer to appear live as it streams in, creating an interactive and visually rich notebook experience.
This is how Neurosurfer bridges retrieval + reasoning, turning your codebase into an explorable knowledge source.

In [8]:
from IPython.display import display, Markdown

# test retrieve
# retrieved_obj = rag_agent.retrieve("What is the main difference between transformers and unsloth?")
# print(retrieved_obj.context)

# test run
llm_response = rag_agent.run("Explain Chunker Configurations in table form.")
response = ""
md_display = display(Markdown(""), display_id=True)
for chunk in llm_response:
    response += chunk
    md_display.update(Markdown(response))

Here's a **table summarizing the key configurations of the `Chunker` class** based on the provided code and documentation:

| **Configuration Parameter**         | **Description**                                                                 | **Default Value**         | **Notes**                                                                 |
|------------------------------------|---------------------------------------------------------------------------------|---------------------------|---------------------------------------------------------------------------|
| `fallback_chunk_size`              | Maximum size (in lines or characters) for fallback chunking strategies.        | `30`                      | Used when no specific strategy is available.                            |
| `overlap_lines`                   | Number of lines to overlap between consecutive chunks for context preservation. | `5`                       | Helps maintain continuity between chunks.                              |
| `char_chunk_size`                 | Maximum size (in characters) for character-based chunking.                     | `1000`                    | Used for prose or generic text content.                                 |
| `comment_block_threshold`         | Minimum number of comment lines to consider a block as comment-only.           | `4`                       | Used for comment-aware filtering.                                      |
| `blacklist_patterns`              | List of regex patterns for files to exclude (e.g., `.git`, `node_modules`, etc.).| As defined in code         | Prevents processing of unwanted file types.                             |
| `line_comment_markers`            | List of comment markers for line-based comment detection (e.g., `#`, `//`).     | `["#"]`                   | Used to identify and filter comment-only blocks.                        |
| `file_extension_handlers`         | Mapping of file extensions to custom chunking strategies.                       | Default registry          | Allows pluggable strategies for specific file types.                    |
| `router_strategy`                 | Strategy for dynamic selection of chunking methods based on content analysis.   | Default router logic       | Prioritizes custom handlers, file extensions, and built-in strategies.  |
| `safety_limits`                   | Safety limits for chunking (e.g., max chunk size, max lines per chunk).         | Default values            | Prevents processing of excessively large or unsafe files.              |
| `use_ast_based_chunking`          | Whether to use AST-based chunking for code files (e.g., Python).                | `True`                    | Enabled for structural awareness in code files.                         |
| `preserve_syntax_structure`       | Whether to preserve syntax structure when chunking code.                        | `True`                    | Ensures code formatting is maintained.                                 |
| `exclude_binary_files`            | Whether to exclude binary files (e.g., images, videos).                         | `True`                    | Prevents processing of non-text files.                                 |
| `max_chunk_lines`                 | Maximum number of lines per chunk for line-based chunking.                      | `100`                     | Controls chunk size for code-like content.                              |
| `min_chunk_lines`                 | Minimum number of lines per chunk to avoid overly small chunks.                | `5`                       | Ensures chunks are meaningful.                                         |
| `chunk_overlap_ratio`             | Ratio of overlap between chunks (e.g., 0.2 for 20% overlap).                    | `0.2`                     | Used for context continuity in line-based chunking.                    |
| `chunking_strategy`               | Primary chunking strategy (e.g., line-based, character-based, AST-based).       | Determined by router      | Selected based on file type and content analysis.                       |

---

### Notes:
- The `Chunker` uses a **priority-based strategy selection**:
  1. **Explicit custom handlers** (registered by name or function)
  2. **Router function results**
  3. **File extension mappings**
  4. **Built-in strategies** (e.g., for `.py`, `.js`, `.md`)
  5. **Heuristic fallback** (line-based or character-based chunking)

- The `blacklist_patterns` and `exclude_binary_files` help avoid processing non-text or unwanted files.

Let me know if you'd like this in a different format (e.g., markdown, JSON, or code comments).

Batches: 100%|██████████| 1/1 [00:00<00:00,  6.37it/s]

INFO     | 2025-11-06 11:51:46 | agent.py:retrieve | [RAGRetriever] Retrieved 6 chunks


## 🧰 Building a Multi-Tool Agent with the Tools Router

This cell demonstrates how to create a **ToolsRouterAgent** — an intelligent agent that can dynamically decide **which tool to use** based on the user’s request.  

Here’s the breakdown:

- [**`BaseTool`**](https://naumanhsa.github.io/neurosurfer/api-reference/tools/base-tool/) is the abstract base every tool inherits from.  
- [**`ToolSpec`**](https://naumanhsa.github.io/neurosurfer/api-reference/tools/tool-spec/) defines the tool’s metadata — including its name, purpose, input parameters, and return type.  
- **`CalculatorTool`** is a simple example tool that performs arithmetic operations with validation and descriptive feedback.  
- **`GeneralQueryAssistantTool`** wraps the LLM itself as a fallback tool for open-ended or natural language queries.  
- [**`Toolkit`**](https://naumanhsa.github.io/neurosurfer/api-reference/tools/toolkit/) bundles multiple tools together, acting as a registry for the agent.  

This agent combines structured tool-calling with LLM reasoning, letting Neurosurfer act as a decision-making orchestrator across multiple specialized tools.

In [9]:
from neurosurfer.agents.tools_router_agent import ToolsRouterAgent
from neurosurfer.tools.toolkit import Toolkit

from neurosurfer.tools.tool_spec import ToolSpec, ToolParam, ToolReturn
from neurosurfer.tools.base_tool import BaseTool, ToolResponse
from neurosurfer.tools.common.general_query_assistant import GeneralQueryAssistantTool

# Simple Calculator Tool
class CalculatorTool(BaseTool):
    spec = ToolSpec(
        name="calculator",
        description="Perform basic arithmetic operations such as addition, subtraction, multiplication, and division.",
        when_to_use="Use this tool when you need to perform basic arithmetic operations.",
        inputs=[
            ToolParam(name="num1", type="float", description="The first number.", required=True),
            ToolParam(name="num2", type="float", description="The second number.", required=True),
            ToolParam(name="operation", type="string", description="The operation to perform: 'add', 'subtract', 'multiply', or 'divide'.", required=True)
        ],
        returns=ToolReturn(type="float", description="The result of the arithmetic operation.")
    )

    def __init__(self, final_answer: bool = False):
        self.final_answer = final_answer

    def __call__(self, num1: float, num2: float, operation: str) -> ToolResponse:
        if operation not in ["add", "subtract", "multiply", "divide"]:
            return ToolResponse(
                final_answer=False,
                observation="Invalid operation. Supported operations are 'add', 'subtract', 'multiply', and 'divide'.",
                extras={}
            )
        
        if operation == "divide" and num2 == 0:
            return ToolResponse(
                final_answer=False,
                observation="Division by zero is not allowed.",
                extras={}
            )
        try:
            num1 = float(num1)
            num2 = float(num2)
            if operation == "add":
                result = num1 + num2
            elif operation == "subtract":
                result = num1 - num2
            elif operation == "multiply":
                result = num1 * num2
            elif operation == "divide":
                result = num1 / num2
        except Exception as e:
            return ToolResponse(
                final_answer=False,
                observation=f"An error occurred: {str(e)}",
                extras={}
            )
        
        return ToolResponse(
            final_answer=self.final_answer,
            observation=float(result),
            extras={}
        )

toolkit = Toolkit(
    tools=[
        CalculatorTool(),
        GeneralQueryAssistantTool(llm=LLM, stream=True)
    ]
)

tools_router_agent = ToolsRouterAgent(
    toolkit=toolkit,
    llm=LLM,
    verbose=True,
    specific_instructions=""
)


INFO     | 2025-11-06 11:55:34 | toolkit.py:register_tool | Registered tool: calculator
INFO     | 2025-11-06 11:55:34 | toolkit.py:register_tool | Registered tool: general_query_assistant


### ⚡ Running the Tools Router Agent

Now we can test the **ToolsRouterAgent** by giving it a natural-language instruction:  


In [ ]:
# Must use Tool Calculator
query = "Perform the calculation 20 * 90"

for chunk in tools_router_agent.run(query, temperature=0.7, max_new_tokens=4000):
    print(chunk, end="")

INFO     | 2025-11-06 11:56:10 | tools_router_agent.py:run | [router] Using tool: calculator
INFO     | 2025-11-06 11:56:10 | tools_router_agent.py:run | [router] Raw inputs: {'num1': 20.0, 'num2': 90.0, 'operation': 'multiply'}
1800.0

In [11]:
# Must use tool for General Queries
query = "Tell me a light-hearted joke!"

for chunk in tools_router_agent.run(query, temperature=0.7, max_new_tokens=4000):
    print(chunk, end="")

INFO     | 2025-11-06 11:57:31 | tools_router_agent.py:run | [router] Using tool: general_query_assistant
INFO     | 2025-11-06 11:57:31 | tools_router_agent.py:run | [router] Raw inputs: {'query': 'Tell me a light-hearted joke!'}
Why don't skeletons fight each other? They don't have the guts!None

## 🧠 Reasoning + Acting: The ReAct Agent

In this cell, we initialize a **ReActAgent**, Neurosurfer’s implementation of the *Reason + Act* paradigm — where the LLM alternates between **thinking** (reasoning in natural language) and **acting** (invoking tools) to solve complex tasks.  

Here’s what happens under the hood:

- [**`toolkit`**](https://naumanhsa.github.io/neurosurfer/api-reference/tools/toolkit) provides access to available tools (like `CalculatorTool` and `GeneralQueryAssistantTool`).  
- [**`LLM`**](https://naumanhsa.github.io/neurosurfer/api-reference/models/chat-models/base-model) handles the reasoning — deciding when to think vs. act.  
- [**`ReActConfig`**](https://naumanhsa.github.io/neurosurfer/api-reference/agents/react-agent/#reactconfig) defines generation settings:
  - `temperature`: creativity in responses.  
  - `allow_input_pruning`: trims irrelevant context for efficiency.  
  - `repair_with_llm`: lets the agent self-correct tool errors.  
  - `verbose`: logs the reasoning and actions step-by-step.  

When you run:
```python
agent.run("Calculate 300 - 300. Then tell me a light-hearted joke about that result.")
```

the agent first reasons about how to approach the query — performs the calculation using the CalculatorTool, observes the result (0), and then calls the LLM to generate a fitting joke.

This cell perfectly demonstrates how Neurosurfer’s ReAct framework blends logic, decision-making, and creativity into one continuous reasoning loop.


In [12]:
from neurosurfer.agents.react import ReActAgent, ReActConfig

react_agent = ReActAgent(
    toolkit=toolkit,
    llm=LLM,
    specific_instructions="Always be concise in your answers. Break the task into steps if needed.",
    config=ReActConfig(
        temperature=0.7,
        max_new_tokens=4096,
        allow_input_pruning=True,
        repair_with_llm=True,
        skip_special_tokens=True,
        verbose=True
    )
)

# print(react_agent._system_prompt())
TASK = """Calculate 300 - 300. Then tell me a light-hearted joke about that result."""

for chunk in react_agent.run(TASK):
    print(chunk, end="")




[🧠] Chain of Thoughts...
Thought: I will first calculate 300 - 300 using the calculator tool, and then use the general_query_assistant to find a light-hearted joke about the result.

Action: {
  "tool": "calculator",
  "inputs": {
    "num1": 300,
    "num2": 300,
    "operation": "subtract"
  },
  "final_answer": false
}

[🔧] Tool: calculator
[📤] Inputs: {'num1': 300, 'num2': 300, 'operation': 'subtract'}

Observation:



[🧠] Chain of Thoughts...
Thought: The result of the calculation is 0. Now, I will use the general_query_assistant to find a light-hearted joke about the result.

Action: {
  "tool": "general_query_assistant",
  "inputs": {
    "query": "Tell me a light-hearted joke about the number 0."
  },
  "final_answer": true
}

[🔧] Tool: general_query_assistant
[📤] Inputs: {'query': 'Tell me a light-hearted joke about the number 0.'}

Why did the number 0 get kicked out of the math class? It kept making a scene every time the teacher said "zero!"

Observation: Why did the number 0 get kicked out of the math class? It kept making a scene every time the teacher 
said "zero!"



[🧠] Chain of Thoughts...
Thought: The calculation result is 0, and the joke provided is appropriate. The task is complete.

Final Answer: The result of 300 - 300 is 0. Here's a light-hearted joke about it: Why did the number 0 get kicked out of the math class? It kept making a scene every time the teacher said "zero!"